<p align="center">
    <img src="https://gitlab.epfl.ch/center-for-imaging/eias-2023-visualization-workshop/-/raw/main/images/stardist_fig.png" height="350" alt="skeletonization image">
</p>

# Cell nuclei detection
---

[StarDist](https://github.com/stardist/stardist) is a deep-learning based Python library used for segmenting star-convex objects, such as cell nuclei, in 2D and 3D images. It is also available as plugins for [ImageJ](https://imagej.net/plugins/stardist), [Napari](https://github.com/stardist/stardist-napari), and [Qupath](https://qupath.readthedocs.io/en/0.3/docs/advanced/stardist.html).

In this notebook, we will use StarDist to detect cell nuclei in an image extracted from the [DeepSlides](https://zenodo.org/record/1184621) public dataset of histopathology images.

### Setup

Check that you have all the necessary packages installed, including `napari` and the `stardist-napari` plugin. If not, you can use the `!` symbol to install them directly from the Jupyter notebook (otherwise, you can use your terminal).

In [11]:
import napari
from stardist.models import StarDist2D

### Get the data

The image we'll use in this tutorial is available for download on [Zenodo](https://zenodo.org/record/8099852) (`deepslide.png`).

In the cell below, we use a Python package called [pooch](https://pypi.org/project/pooch/) to automatically download the image from Zenodo into the **data** folder of this repository.

In [12]:
import pooch
from pathlib import Path

data_path = Path('.').resolve().parent / 'data'
fname = 'deepslide.png'

pooch.retrieve(
    url="https://zenodo.org/record/8099852/files/deepslide.png",
    known_hash="md5:67d2dac6f327e2d3749252d46799861a",
    path=data_path,
    fname=fname,
    progressbar=True,
)

print(f'Downloaded image {fname} into: {data_path}')

Downloaded image deepslide.png into: /home/wittwer/code/eias-2023-visualization-workshop/data


### Read the image

We use the `imread` function from Scikit-image to read our PNG image.

In [13]:
from skimage.io import imread

image = imread(data_path / 'deepslide.png')

print(f'Loaded image in an array of shape: {image.shape} and data type {image.dtype}')
print(f'Intensity range: [{image.min()} - {image.max()}]')

Loaded image in an array of shape: (513, 513, 3) and data type uint8
Intensity range: [3 - 245]


If you run into troubles, don't hesitate to ask for help 🤚🏽.

### Load the image into Napari

Let's open a viewer and load our image to have a look at it.

In [14]:
viewer = napari.Viewer()
viewer.add_image(image, name="H&E (DeepSlides)")

_plugin_manager.py (542): Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten


<Image layer 'H&E (DeepSlides)' at 0x7f825e2cac70>

### Intensity normalization

Let's rescale our image to the range 0-1. By doing so, it is also converted to an array of data type `float`.

In [15]:
from skimage.exposure import rescale_intensity

image_normed = rescale_intensity(image, out_range=(0, 1))

print(f'Intensity range: [{image_normed.min()} - {image_normed.max()}]')
print(f'Array type: {image_normed.dtype}')

Intensity range: [0.0 - 1.0]
Array type: float64


### Instantiate a StarDist model

The StarDist developers provide a few pre-trained models that may already be applied to suitable images.

Here, we will use the *Versatile (H&E nuclei)* model that was trained on images from the MoNuSeg 2018 training data and the TNBC dataset from Naylor et al. (2018).

In [16]:
model = StarDist2D.from_pretrained("2D_versatile_he")

model

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


StarDist2D(2D_versatile_he): YXC → YXC
├─ Directory: None
└─ Config2D(n_dim=2, axes='YXC', n_channel_in=3, n_channel_out=33, train_checkpoint='weights_best.h5', train_checkpoint_last='weights_last.h5', train_checkpoint_epoch='weights_now.h5', n_rays=32, grid=(2, 2), backbone='unet', n_classes=None, unet_n_depth=3, unet_kernel_size=[3, 3], unet_n_filter_base=32, unet_n_conv_per_depth=2, unet_pool=[2, 2], unet_activation='relu', unet_last_activation='relu', unet_batch_norm=False, unet_dropout=0.0, unet_prefix='', net_conv_after_unet=128, net_input_shape=[None, None, 3], net_mask_shape=[None, None, 1], train_shape_completion=False, train_completion_crop=32, train_patch_size=[512, 512], train_background_reg=0.0001, train_foreground_only=0.9, train_sample_cache=True, train_dist_loss='mae', train_loss_weights=[1, 0.1], train_class_weights=(1, 1), train_epochs=200, train_steps_per_epoch=200, train_learning_rate=0.0003, train_batch_size=8, train_n_val_patches=3, train_tensorboard=True, train_r

### Run the model

We use the `predict_instances` method of the model to generate a segmenation mask (`labels`) and a representation of the cells as polygons (`polys`).

In [17]:
labels, polys = model.predict_instances(
    image_normed,  # The image must be normalized!
    axes="YXC",
    prob_thresh=0.5,  # Detection probability threshold
    nms_thresh=0.1,  # Remove detections overlapping by more than this threshold
    scale=1,  # Higher values are suitable for lower resolution data
    return_labels=True,
)

# We also get detection probabilities:
probabilities = list(polys["prob"])

n_detections = len(probabilities)

print(f'{n_detections} cells detected.')

1/1 [==============================] - 0s 226ms/step
119 cells detected.


### Visualization in Napari

In [18]:
import numpy as np
import matplotlib.pyplot as plt

# Create a custom color lookup table based on detection probabilities
probas_incl_bg = np.zeros(n_detections + 1)  # Include a value for the background
probas_incl_bg[1:] = probabilities

colors = plt.cm.get_cmap('inferno')(probas_incl_bg)  # Convert probability values to colors
colors[0, -1] = 0.0  # Make the background transparent (alpha channel = 0)
colormap = dict(zip(np.arange(len(probas_incl_bg)), colors))

labels_layer = viewer.add_labels(
    labels, 
    name='Segmentation', 
    color=colormap,
    properties={'probabilities': probas_incl_bg},
    opacity=0.7
)

# In Napari, you can display some text in the top-left part of the window, for example:
viewer.text_overlay.visible = True
viewer.text_overlay.text = f'Number of detections: {n_detections}'

1201456626.py (8): The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.


### The `stardist-napari` plugin

In fact, a Napari [plugin for StarDist](https://github.com/stardist/stardist-napari) already exists. If you have installed it using `pip install stardist-napari`, you should be able to find it in the `Plugins` menu of Napari.

You can check that the segmentation produced by the plugin is similar to what you got by running the model in this notebook!

<p align="center">
    <img src="https://gitlab.epfl.ch/center-for-imaging/eias-2023-visualization-workshop/-/raw/main/images/stardist_plugin_fig.png" height="350" alt="StarDist plugin">
</p>

<p align="center">
    <a href="https://gitlab.epfl.ch/center-for-imaging/eias-2023-visualization-workshop/-/blob/main/examples/README.md">🔙 Back to case studies</a>
</p>
